In [1]:
import sys
sys.path.append('/Users/justinalsing/Dropbox/science/pydelfi-tf2/pydelfi/pydelfi')

import numpy as np
import matplotlib.pyplot as plt
import getdist
from getdist import plots, MCSamples
import priors as priors
import ndes as ndes
import delfi as delfi
import tensorflow as tf
import simulators.cosmic_shear.cosmic_shear as cosmic_shear
import pickle
import score as score
#tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

/usr/local/lib/python3.7/site-packages/getdist/plots.py:11: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('Agg')


In [2]:
### SET UP THE SIMULATOR ###

# Set up the tomography simulations
CosmicShearSimulator = cosmic_shear.TomographicCosmicShear(pz = pickle.load(open('simulators/cosmic_shear/pz_5bin.pkl', 'rb')),
                                                           lmin = 10, lmax = 1000, n_ell_bins = 5, 
                                                           sigma_e = 0.3, nbar = 30, Area = 15000)

# Simulator function: This must be of the form simulator(theta, seed, args) -> simulated data vector
def simulator(theta, seed, simulator_args, batch=1):
    return CosmicShearSimulator.simulate(theta, seed)

# Simulator arguments
simulator_args = None

In [3]:
### SET UP THE PRIOR ###

# Define the priors parameters
lower = np.array([0, 0.4, 0, 0.4, 0.7])
upper = np.array([1, 1.2, 0.1, 1.0, 1.3])
prior_mean = np.array([0.3, 0.8, 0.05, 0.70, 0.96])
prior_covariance = np.eye(5)*np.array([0.1, 0.1, 0.05, 0.3, 0.3])**2

# Prior
prior = priors.TruncatedGaussian(prior_mean, prior_covariance, lower, upper)

In [4]:
### SET UP THE COMPRESSOR ###

# Fiducial parameters
theta_fiducial = np.array([0.3, 0.8, 0.05, 0.70, 0.96])

# Expected support of Wishart likelihood (fiducial inverse power spectrum)
C = CosmicShearSimulator.power_spectrum(theta_fiducial)
Cinv = np.array([np.linalg.inv(C[l,:,:]) for l in range(CosmicShearSimulator.n_ell_bins)])

# Degrees of freedom (effective number of modes per band power)
nl = CosmicShearSimulator.nl

# Calculate derivatives of the expected power spectrum
step = np.array(abs(theta_fiducial)*np.array([0.05, 0.05, 0.05, 0.05, 0.05]))
dCdt = CosmicShearSimulator.compute_derivatives(theta_fiducial, step)

# Define compression as score-MLE of a Wishart likelihood
Compressor = score.Wishart(theta_fiducial, nl, Cinv, dCdt, prior_mean=prior_mean, prior_covariance=prior_covariance)

# Pull out Fisher matrix inverse
Finv = Compressor.Finv

# Compressor function: This must have the form compressor(data, args) -> compressed summaries (pseudoMLE)
def compressor(d, compressor_args):
    return Compressor.scoreMLE(d)
compressor_args = None

In [5]:
### GENERATE MOCK DATA VECTOR ###

seed = 0
data = simulator(theta_fiducial, seed, simulator_args)
compressed_data = compressor(data, compressor_args)

In [6]:
# Create an ensemble of NDEs
NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=5, n_data=5, n_hidden=[50,50], n_mades=5, activations=[tf.tanh, tf.tanh]),
        ndes.MixtureDensityNetwork(n_parameters=5, n_data=5, n_components=1, n_hidden=[30,30], activations=[tf.tanh, tf.tanh]),
        ndes.MixtureDensityNetwork(n_parameters=5, n_data=5, n_components=2, n_hidden=[30,30], activations=[tf.tanh, tf.tanh]),
        ndes.MixtureDensityNetwork(n_parameters=5, n_data=5, n_components=3, n_hidden=[30,30], activations=[tf.tanh, tf.tanh]),
        ndes.MixtureDensityNetwork(n_parameters=5, n_data=5, n_components=4, n_hidden=[30,30], activations=[tf.tanh, tf.tanh]),
        ndes.MixtureDensityNetwork(n_parameters=5, n_data=5, n_components=5, n_hidden=[30,30], activations=[tf.tanh, tf.tanh])]


# Create the DELFI object
DelfiEnsemble = delfi.Delfi(compressed_data, prior, NDEs, Finv=Finv, theta_fiducial=theta_fiducial, 
                       param_limits = [lower, upper],
                       param_names = ['\Omega_m', 'S_8', '\Omega_b', 'h', 'n_s'], 
                       results_dir = "simulators/cosmic_shear/results/",
                       input_normalization='fisher')

In [9]:
# Do the Fisher pre-training
DelfiEnsemble.fisher_pretraining(mode="samples")

KeyboardInterrupt: 

In [ ]:
# Initial samples, batch size for population samples, number of populations
n_initial = 200
n_batch = 200
n_populations = 39

# Do the SNL training
DelfiEnsemble.sequential_training(simulator, compressor, n_initial, n_batch, n_populations, patience=10, save_intermediate_posteriors=True)

In [8]:
NDEs[0].trainable_variables

[<tf.Variable 'made_1/W0:0' shape=(5, 50) dtype=float32, numpy=
 array([[ 5.23645341e-01, -1.39562041e-01, -2.14223519e-01,
          1.10194576e+00,  4.69015092e-01,  4.05186743e-01,
          3.17452326e-02,  1.44296706e-01, -1.84002146e-02,
         -3.34744245e-01,  5.17341134e-04,  1.16310827e-01,
         -2.51132697e-01, -9.68773067e-02,  4.12629724e-01,
         -3.10207903e-01,  3.95923972e-01,  3.01327914e-01,
          2.73814023e-01, -3.75052273e-01,  5.11782587e-01,
         -2.37160593e-01,  3.74227762e-01,  2.41026487e-02,
          2.49731749e-01,  4.82368797e-01, -3.97745408e-02,
          4.43218619e-01, -7.91396648e-02, -3.62407267e-01,
          4.79250044e-01, -2.29393337e-02,  1.85456410e-01,
         -6.85495958e-02, -3.18168283e-01, -4.00384098e-01,
          1.90248474e-01,  7.53791630e-02,  4.50087249e-01,
          5.41565299e-01, -5.52384853e-01,  3.76862228e-01,
          3.65062147e-01,  1.56490564e-01, -1.92277402e-01,
         -1.01331985e+00,  9.1640132